In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.conf import SparkConf
config = SparkConf()
# config.set("property", "value")
config.setMaster("local[4]").setAppName("WindowFunction")

from pyspark.sql import SparkSession
# spark Session, entry point for Spark SQL, DataFrame
spark = SparkSession.builder\
                    .config(conf=config)\
                    .getOrCreate()

sc = spark.sparkContext

22/05/10 00:07:38 WARN Utils: Your hostname, ubuntu-virtual-machine resolves to a loopback address: 127.0.1.1; using 192.168.174.129 instead (on interface ens33)
22/05/10 00:07:38 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/05/10 00:07:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/10 00:07:43 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/05/10 00:07:43 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/05/10 00:07:43 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/05/10 00:07:43 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
2

In [3]:
data = [ ("James", "Sales", 3000),
    ("Michael", "Sales", 4600),
    ("Robert", "Sales", 4100),
    ("Maria", "Finance", 3000),
    ("James", "Sales", 3000),
    ("Scott", "Finance", 3300),
    ("Jen", "Finance", 3900),
    ("Jeff", "Marketing", 3000),
    ("Kumar", "Marketing", 2000),
    ("Saif", "Sales", 4100)
   ]

empDf = spark.createDataFrame(data=data, schema=['name', 'dept', 'salary'])
empDf.printSchema()
empDf.show()

empDf.rdd.getNumPartitions()

root
 |-- name: string (nullable = true)
 |-- dept: string (nullable = true)
 |-- salary: long (nullable = true)



+-------+---------+------+
|   name|     dept|salary|
+-------+---------+------+
|  James|    Sales|  3000|
|Michael|    Sales|  4600|
| Robert|    Sales|  4100|
|  Maria|  Finance|  3000|
|  James|    Sales|  3000|
|  Scott|  Finance|  3300|
|    Jen|  Finance|  3900|
|   Jeff|Marketing|  3000|
|  Kumar|Marketing|  2000|
|   Saif|    Sales|  4100|
+-------+---------+------+



4

In [4]:
empDf.rdd.glom().collect()

[[Row(name='James', dept='Sales', salary=3000),
  Row(name='Michael', dept='Sales', salary=4600)],
 [Row(name='Robert', dept='Sales', salary=4100),
  Row(name='Maria', dept='Finance', salary=3000)],
 [Row(name='James', dept='Sales', salary=3000),
  Row(name='Scott', dept='Finance', salary=3300)],
 [Row(name='Jen', dept='Finance', salary=3900),
  Row(name='Jeff', dept='Marketing', salary=3000),
  Row(name='Kumar', dept='Marketing', salary=2000),
  Row(name='Saif', dept='Sales', salary=4100)]]

In [5]:
empDf.write.option("header", True)\
  .partitionBy("dept")\
  .csv("/home/ubuntu/employees")

In [6]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

# specification for window, partitions, functions that should be applied on partition
# with in department, order the data based on salary in ascending order
windowSpec = Window.partitionBy("dept").orderBy("salary")
# we have apply the spec on dataframe
df = empDf.withColumn("row_number", row_number().over(windowSpec))

df.printSchema()
df.show()

root
 |-- name: string (nullable = true)
 |-- dept: string (nullable = true)
 |-- salary: long (nullable = true)
 |-- row_number: integer (nullable = true)



+-------+---------+------+----------+
|   name|     dept|salary|row_number|
+-------+---------+------+----------+
|  James|    Sales|  3000|         1|
|  James|    Sales|  3000|         2|
| Robert|    Sales|  4100|         3|
|   Saif|    Sales|  4100|         4|
|Michael|    Sales|  4600|         5|
|  Maria|  Finance|  3000|         1|
|  Scott|  Finance|  3300|         2|
|    Jen|  Finance|  3900|         3|
|  Kumar|Marketing|  2000|         1|
|   Jeff|Marketing|  3000|         2|
+-------+---------+------+----------+



In [7]:
from pyspark.sql.window import Window
from pyspark.sql.functions import rank

# rank with gap with ascending order
"""
score  rank
90      1
90      1
89      3  [gap, 2 not included]
"""
windowSpec = Window.partitionBy("dept").orderBy("salary")

df = empDf.withColumn("rank", rank().over(windowSpec))
df.show()

+-------+---------+------+----+
|   name|     dept|salary|rank|
+-------+---------+------+----+
|  James|    Sales|  3000|   1|
|  James|    Sales|  3000|   1|
| Robert|    Sales|  4100|   3|
|   Saif|    Sales|  4100|   3|
|Michael|    Sales|  4600|   5|
|  Maria|  Finance|  3000|   1|
|  Scott|  Finance|  3300|   2|
|    Jen|  Finance|  3900|   3|
|  Kumar|Marketing|  2000|   1|
|   Jeff|Marketing|  3000|   2|
+-------+---------+------+----+



In [8]:
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, desc

# rank with gap
"""
score  rank
90      1
90      1
89      3  [gap, 2 not included]
"""
windowSpec = Window.partitionBy("dept").orderBy(desc("salary"))

df = empDf.withColumn("rank", rank().over(windowSpec))
df.show()

+-------+---------+------+----+
|   name|     dept|salary|rank|
+-------+---------+------+----+
|Michael|    Sales|  4600|   1|
| Robert|    Sales|  4100|   2|
|   Saif|    Sales|  4100|   2|
|  James|    Sales|  3000|   4|
|  James|    Sales|  3000|   4|
|    Jen|  Finance|  3900|   1|
|  Scott|  Finance|  3300|   2|
|  Maria|  Finance|  3000|   3|
|   Jeff|Marketing|  3000|   1|
|  Kumar|Marketing|  2000|   2|
+-------+---------+------+----+



In [9]:
from pyspark.sql.window import Window
from pyspark.sql.functions import dense_rank, desc

# dense_rank ranking without gap
"""
score  rank
90      1
90      1
89      2  
"""
windowSpec = Window.partitionBy("dept").orderBy(desc("salary"))

df = empDf.withColumn("rank", dense_rank().over(windowSpec))
df.show()

+-------+---------+------+----+
|   name|     dept|salary|rank|
+-------+---------+------+----+
|Michael|    Sales|  4600|   1|
| Robert|    Sales|  4100|   2|
|   Saif|    Sales|  4100|   2|
|  James|    Sales|  3000|   3|
|  James|    Sales|  3000|   3|
|    Jen|  Finance|  3900|   1|
|  Scott|  Finance|  3300|   2|
|  Maria|  Finance|  3000|   3|
|   Jeff|Marketing|  3000|   1|
|  Kumar|Marketing|  2000|   2|
+-------+---------+------+----+



In [10]:
from pyspark.sql.window import Window
from pyspark.sql.functions import percent_rank, desc

# percent_rank ranking with perecent calculation
"""
 
"""
windowSpec = Window.partitionBy("dept").orderBy(desc("salary"))

df = empDf.withColumn("rank", percent_rank().over(windowSpec))
df.show()

+-------+---------+------+----+
|   name|     dept|salary|rank|
+-------+---------+------+----+
|Michael|    Sales|  4600| 0.0|
| Robert|    Sales|  4100|0.25|
|   Saif|    Sales|  4100|0.25|
|  James|    Sales|  3000|0.75|
|  James|    Sales|  3000|0.75|
|    Jen|  Finance|  3900| 0.0|
|  Scott|  Finance|  3300| 0.5|
|  Maria|  Finance|  3000| 1.0|
|   Jeff|Marketing|  3000| 0.0|
|  Kumar|Marketing|  2000| 1.0|
+-------+---------+------+----+



In [11]:
from pyspark.sql.window import Window
from pyspark.sql.functions import ntile, desc

# ntile ranking with related certain range for range
# rank shall fit into a range
"""
 
"""
windowSpec = Window.partitionBy("dept").orderBy(desc("salary"))

df = empDf.withColumn("rank", ntile(3).over(windowSpec))
df.show()

+-------+---------+------+----+
|   name|     dept|salary|rank|
+-------+---------+------+----+
|Michael|    Sales|  4600|   1|
| Robert|    Sales|  4100|   1|
|   Saif|    Sales|  4100|   2|
|  James|    Sales|  3000|   2|
|  James|    Sales|  3000|   3|
|    Jen|  Finance|  3900|   1|
|  Scott|  Finance|  3300|   2|
|  Maria|  Finance|  3000|   3|
|   Jeff|Marketing|  3000|   1|
|  Kumar|Marketing|  2000|   2|
+-------+---------+------+----+

